In [ ]:
import torch
from typing import List, Tuple
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "Qwen/Qwen3-Reranker-0.6B"

# ===== Инициализация модели один раз =====
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="left",
)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device).eval()

# Токены для "классов" yes/no
TOKEN_TRUE_ID = tokenizer.convert_tokens_to_ids("yes")
TOKEN_FALSE_ID = tokenizer.convert_tokens_to_ids("no")

# Префикс/суффикс — как в оф. примере
PREFIX = (
    "<|im_start|>system\n"
    "Judge whether the Document meets the requirements based on the Query and the Instruct provided. "
    "Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n"
    "<|im_start|>user\n"
)
SUFFIX = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"

MAX_LEN = 8192
PREFIX_TOKENS = tokenizer.encode(PREFIX, add_special_tokens=False)
SUFFIX_TOKENS = tokenizer.encode(SUFFIX, add_special_tokens=False)


def _build_inputs(
    query: str,
    docs: List[str],
    instruction: str,
) -> dict:
    """
    Готовим батч токенов под Qwen3-Reranker.
    """
    # Строим текст для каждого (query, doc)
    pairs = [
        f"<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}"
        for doc in docs
    ]

    # Сначала токенизируем без префикса/суффикса
    tokenized = tokenizer(
        pairs,
        padding=False,
        truncation="longest_first",
        return_attention_mask=False,
        max_length=MAX_LEN - len(PREFIX_TOKENS) - len(SUFFIX_TOKENS),
    )

    # Добавляем префикс/суффикс токенами
    for i, ids in enumerate(tokenized["input_ids"]):
        tokenized["input_ids"][i] = PREFIX_TOKENS + ids + SUFFIX_TOKENS

    # Паддинг до общего размера
    tokenized = tokenizer.pad(
        tokenized,
        padding=True,
        max_length=MAX_LEN,
        return_tensors="pt",
    )

    # На нужное устройство
    for key in tokenized:
        tokenized[key] = tokenized[key].to(device)

    return tokenized


@torch.no_grad()
def qwen3_rerank(
    query: str,
    docs: List[str],
    instruction: str = (
        "Given a web search query, retrieve relevant passages that answer the query"
    ),
    top_k: int | None = None,
) -> List[Tuple[int, str, float]]:
    """
    Реранкинг документов под один запрос.

    Возвращает список (original_index, doc, score),
    отсортированный по score по убыванию.
    """
    if not docs:
        return []

    inputs = _build_inputs(query, docs, instruction)

    # Берём logits последнего токена
    logits = model(**inputs).logits[:, -1, :]

    # Logits для "yes" и "no"
    true_logits = logits[:, TOKEN_TRUE_ID]
    false_logits = logits[:, TOKEN_FALSE_ID]

    # Превращаем в вероятность "yes" через softmax
    stacked = torch.stack([false_logits, true_logits], dim=1)
    probs = torch.softmax(stacked, dim=1)[:, 1]  # P("yes")
    scores = probs.tolist()

    # Сортируем документы по убыванию score
    ranked = sorted(
        enumerate(zip(docs, scores)),
        key=lambda x: x[1][1],
        reverse=True,
    )

    if top_k is not None:
        ranked = ranked[:top_k]

    # Приводим к формату (orig_idx, doc, score)
    return [(idx, doc, score) for idx, (doc, score) in ranked]

In [ ]:
query = "Как работает градиентный бустинг?"
docs = [
    "Градиентный бустинг — ансамблевый метод, который добавляет деревья итеративно.",
    "Линейная регрессия — простая модель с одной матрицей весов.",
]

results = qwen3_rerank(query, docs, top_k=None)
for orig_idx, doc, score in results:
    print(f"{score:.4f} | #{orig_idx}: {doc}")